In [85]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [90]:
train_df = pd.read_csv('/Users/taraapple/Desktop/train.csv')
print(train_df.shape)

(42000, 785)


In [91]:
test_df = pd.read_csv('/Users/taraapple/Desktop/test.csv')
print(test_df.shape)

(28000, 784)


In [95]:
y_train = train_df['label'].values
X_train = train_df.drop(['label'],1).values



In [98]:
test_df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
y_test = test_df['label'].values
#X_test = test_df.drop(['label'],1).values

KeyError: 'label'

In [4]:
print(y_test.shape)

(6300,)


In [5]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [7]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [8]:
fit(mlp, train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 8.159407	 Accuracy:18.750%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.227379	 Accuracy:69.792%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.521355	 Accuracy:77.970%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.309238	 Accuracy:81.250%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.544148	 Accuracy:83.520%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.220697	 Accuracy:84.935%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.367417	 Accuracy:85.725%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.654621	 Accuracy:86.565%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.742725	 Accuracy:87.149%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.252580	 Accuracy:87.694%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.203498	 Accuracy:88.074%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.285547	 Accuracy:88.561%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.091625	 Accuracy:88.878%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.561206	 Accuracy:89.113%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.442160	 Accuracy:89.279%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.143203	 Accuracy:89.531%
Epoch

In [9]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))
evaluate(mlp)

Test accuracy:0.955% 


In [10]:
torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([35700, 1, 28, 28])
torch.Size([6300, 1, 28, 28])


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [12]:
fit(cnn,train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 24.555870	 Accuracy:9.375%
Epoch : 0 [1600/35700 (4%)]	Loss: 2.077325	 Accuracy:15.625%
Epoch : 0 [3200/35700 (9%)]	Loss: 1.375981	 Accuracy:25.557%
Epoch : 0 [4800/35700 (13%)]	Loss: 1.094583	 Accuracy:36.796%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.967033	 Accuracy:45.072%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.404181	 Accuracy:51.594%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.678731	 Accuracy:55.845%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.598726	 Accuracy:59.740%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.935075	 Accuracy:62.765%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.520798	 Accuracy:65.195%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.327560	 Accuracy:67.278%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.428962	 Accuracy:69.215%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.450609	 Accuracy:70.825%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.256112	 Accuracy:72.206%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.646318	 Accuracy:73.337%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.175919	 Accuracy:74.451%
Epoch

In [13]:
evaluate(cnn) 

Test accuracy:0.952% 


# MY Neural Network

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import utils 
import matplotlib.pyplot as plt
import numpy as np
from numpy import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [101]:
df = pd.read_csv('/Users/taraapple/Desktop/train.csv')
print(df.shape)

(42000, 785)


In [106]:
df = pd.read_csv('/Users/taraapple/Desktop/test.csv')
print(df.shape)

(28000, 784)


In [107]:
df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
X = df.drop(['label'],1).values

In [104]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [105]:
y_train

array([1, 0, 1, ..., 7, 6, 9])

In [35]:
X = df.drop(['label'],1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [10]:
print(X_train.shape)
print(y_train.shape)

(35700, 784)
(35700,)


In [38]:
X_train=X_train.reshape(35700,28,28)
X_train.shape

(35700, 28, 28)

In [11]:
print(X_test.shape)
print(y_test.shape)

(6300, 784)
(6300,)


In [42]:
X_test=X_test.reshape(6300,28,28)
X_train.shape

(35700, 28, 28)

In [60]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
print(type(X_train_tensor))
print(type(X_test_tensor))


<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [61]:
# add a dimension to tensor to become B*C*H*W 
if len(X_train.shape)==3:
    X_train_tensor = X_train_tensor.unsqueeze(1) 
print(X_train_tensor.shape)
if len(X_test.shape)==3: 
    X_test_tensor = X_test_tensor.unsqueeze(1)
print(X_test_tensor.shape)

#torch.Size([35700, 1, 28, 28]) 
#torch.Size([6300, 1, 28, 28])

torch.Size([35700, 1, 28, 28])
torch.Size([6300, 1, 28, 28])


In [62]:
BATCH_SIZE = 64

## transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

In [73]:
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [74]:
from torch.utils.data import TensorDataset
# wrap tensors into a dataset
train_ds = TensorDataset(X_train_tensor, y_train_tensor) 
test_ds = TensorDataset(X_test_tensor, y_test_tensor)
for x,y in train_ds: 
    print(x.shape,y.item()) 
    break

torch.Size([1, 28, 28]) 5


In [76]:
from torch.utils.data import DataLoader
# create a data loader from dataset
train_dl = DataLoader(train_ds, batch_size=8) 
test_dl = DataLoader(test_ds, batch_size=8)
            # iterate over batches
for xb,yb in train_dl:
    print(xb.shape) 
    print(yb.shape) 
    break

torch.Size([8, 1, 28, 28])
torch.Size([8])


In [77]:
from torch import nn
# define a two-layer model 
model = nn.Sequential(
    nn.Linear(4, 5), 
    nn.ReLU(), 
    nn.Linear(5, 1),
            )
print(model)

Sequential(
  (0): Linear(in_features=4, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)


In [81]:
print(next(model.parameters()).device)


cpu


In [83]:
device = torch.device("cuda:0") 
model.to(device)
print(next(model.parameters()).device) 

AssertionError: Torch not compiled with CUDA enabled